In [ ]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [ ]:
filepath = Path("../sample_files/lvhexmesh_tetcoarse_counterpart.vtk") 
lv_src = LV.from_pyvista_read(filepath)

filepath = Path("../sample_files/lvhexmesh.vtk") 
lv_dst = LV.from_pyvista_read(filepath)

In [ ]:
# offset = 0
# for i, key in enumerate([LV_FIBERS.F0, LV_FIBERS.S0, LV_FIBERS.N0]):
#   shape = lv_src.get(GEO_DATA.MESH_POINT_DATA, key).shape
#   dd = d[:, offset:offset+shape[-1]]
#   offset += shape[-1]
#   print(dd.shape)

In [ ]:
d = lv_src.get(GEO_DATA.MESH_CELL_DATA, [LV_FIBERS.F0, LV_FIBERS.S0, LV_FIBERS.N0])

In [ ]:
to_regress = [LV_FIBERS.F0, LV_FIBERS.S0, LV_FIBERS.N0]
lv_dst.regress_from_other(lv_src, 
                          to_regress
                          )

In [ ]:
lv_dst.identify_base_and_apex_surfaces() # computes normal
lv_dst.compute_fiber_angles()

In [ ]:
lv_dst.plot_streamlines(LV_FIBERS.F0, scalars=LV_FIBERS.F0_ANGLES, decimate_boundary=0.8)

In [ ]:
lv_dst.mesh.save("../sample_files/lvhex_with_fibers_test.vtk")

In [ ]:
X = lv_src.points()
Y = lv_src.get(GEO_DATA.MESH_POINT_DATA, LV_FIBERS.F0)
XI = lv_dst.points()

In [ ]:
Y.shape

In [ ]:
Y1 = lv_src.get(GEO_DATA.MESH_POINT_DATA, LV_FIBERS.F0)
Y2 = lv_src.get(GEO_DATA.MESH_POINT_DATA, LV_FIBERS.S0)
Y3 = lv_src.get(GEO_DATA.MESH_POINT_DATA, LV_FIBERS.N0)
Y = np.hstack((Y1, Y2, Y3))
Y.shape

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

regr = MultiOutputRegressor(GradientBoostingRegressor(random_state=0, 
                                                      subsample=0.75,
                                                      n_estimators=200)).fit(X, Y)
YI = regr.predict(XI)

In [ ]:
from sklearn.neural_network import MLPRegressor
MLPR = MLPRegressor((100,),
                    early_stopping=True,
                    validation_fraction=0.25,
                    )
MLPR = MLPR.fit(X, Y)
YI2 = MLPR.predict(XI)

In [ ]:
lv_dst.mesh.point_data[LV_FIBERS.F0.value] = YI[:, :3]
lv_dst.mesh.point_data[LV_FIBERS.S0.value] = YI[:, 3:6]
lv_dst.mesh.point_data[LV_FIBERS.S0.value] = YI[:, 6:]

In [ ]:
lv_dst.mesh.point_data[LV_FIBERS.F0.value] = YI2[:, :3]
lv_dst.mesh.point_data[LV_FIBERS.S0.value] = YI2[:, 3:6]
lv_dst.mesh.point_data[LV_FIBERS.S0.value] = YI2[:, 6:]

In [ ]:
lv_dst.identify_base_and_apex_surfaces() # computes normal
lv_dst.compute_fiber_angles()

In [ ]:
lv_dst.plot_streamlines(LV_FIBERS.F0, scalars=LV_FIBERS.F0_ANGLES, decimate_boundary=0.8)

In [ ]:
lv_src.plot_streamlines(LV_FIBERS.F0, scalars=LV_FIBERS.F0_ANGLES, decimate_boundary=0.8)

In [ ]:
lv_dst.mesh.save("./testhexfibers.vtk")